## Contents
1. 데이터셋 확인
- 1.1 Null data check
- 1.2 Target label 확인
2. Exploratory data analysis
- 2.1 Pclass
- 2.2 Sex
- 2.3 Both Sex and Pclass
- 2.4 Age
- 2.5 Pclass, Sex, Age
- 2.6 Embarked
- 2.7 Family - SibSp(형제 자매) + Parch(부모, 자녀)
- 2.8 Cabin

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('seaborn')
sns.set(font_scale=2.5) # 이 두 줄은 본 필자가 항상 쓰는 방법이다. matplotlib의 기본 scheme 말고 seaborn scheme을 세팅하고, 일일이 graph의 font size를 지정할 필요 없이 seaborn의 font_scale을 사용하면 편하다.

import missingno as msno

#ignore warnings
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

![](https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FXgPmJ%2FbtqyRNzEs5v%2FoEuIJgkbIjCkgutPKwBCSk%2Fimg.png)  

>missingno
- missingno는 결측치를 파악하는데 직관적인 도움을 주는 패키지다.  

>%matplotlib inline
- IPython에서 제공하는 Rich output에 대한 표현 방식 (Rich output : 그림, 소리, 애니메이션 등)
- notebook을 실행한 브라우저에서 바로 그림을 볼 수 있게 해준다.

앞으로 우리가 해야할 프로세스는 대략 아래와 같다.
1. 데이터셋 확인 - 대부분의 캐글 데이터들은 잘 정제되어 있다. 하지만 가끔 null data가 존재한다. 이를 확인하고, 향후 수정한다. 
2. 탐색적 데이터 분석(exploratory data analysis) - 여러 feature들을 개별적으로 분석하고, feature들 간의 상관관계를 확인한다. 여러 시각화 툴을 사용해 insight를 얻는다. 
3. feature engineering - 모델을 세우기에 앞서, 모델의 성능을 높일 수 있도록 feature들을 engineering한다. ont-hot encoding, class로 나누기, 구간으로 나누기, 텍스트 데이터 처리 등을 한다.
4. model 만들기 - sklearn을 사용해 모델을 만든다. 파이썬에서 머신 러닝을 할 때는 sklearn을 사용하면 수많은 알고리즘을 일관된 문법으로 사용할 수 있다. 물론 딥러닝을 위해 tensorflow, pytorch 등을 사용할 수도 있다. 
5. 모델 학습 및 예측 - trainset을 가지고 모델을 학습시킨 후, testset을 가지고 prediction한다. 
6. 모델 평가 - 예측 성능이 원하는 수준인지 판단한다. 풀려는 문제에 따라 모델을 평가하는 방식도 달라진다. 학습된 모델이 어떤 것을 학습하였는지 확인해본다.   

<br>

## 1. Dataset 확인
- 파이썬에서 테이블화 된 데이터를 다루는 데 가장 최적화되어 있으며, 많이 쓰이는 라이브러리는 pandas다. 
- 우리는 pandas를 사용해 데이터셋의 간단한 통계적 분석부터, 복잡한 처리들을 간단한 메소드를 사용해 해낼 수 있다. 
- 파이썬으로 데이터분석을 한다고 하면 반드시 능숙해져야 할 라이브러리이니, 여러 커널들을 공부하면서 사용법에 익숙해지도록 반복하자. 
- 캐글에서 데이터셋은 보통 train, testset으로 나뉘어 있다. 

In [2]:
df_train = pd.read_csv('../input/titanic/train.csv')
df_test = pd.read_csv('../input/titanic/test.csv')

In [3]:
df_train.head()

In [4]:
df_train.describe()

In [5]:
df_test.describe()

- 테이블에서 보다시피, PassengerID 숫자와 다른, 그러니까 null data가 존재하는 열(feature)가 있는 것 같다. 
- 이를 좀 더 보기 편하도록 그래프로 시각화해서 살펴보자.

## 1.1 Null data check

In [6]:
for col in df_train.columns:
    msg = 'column: {:>10}\t Percent of NaN value: {:.2f}%'.format(col, 100 * (df_train[col].isnull().sum() / df_train[col].shape[0]))
    print(msg)

In [7]:
for col in df_test.columns:
    msg = 'column: {:>10}\t Percent of NaN value: {:.2f}%'.format(col, 100 * (df_test[col].isnull().sum() / df_test[col].shape[0]))
    print(msg)

- Train, Test set에서 Age(둘 다 약 20%), Cabin(둘 다 약 80%), Embarked(Train만 0.22%), null data 존재하는 것을 볼 수 있다. 
- MSNO라는 라이브러리를 사용하면 null data의 존재를 더 쉽게 볼 수 있다. 

In [8]:
msno.matrix(df=df_train.iloc[:, :], figsize=(8, 8), color=(0.8, 0.5, 0.2))

In [9]:
msno.bar(df=df_train.iloc[:, :], figsize=(8, 8), color=(0.8, 0.5, 0.2))

In [10]:
msno.bar(df=df_test.iloc[:, :], figsize=(8, 8), color=(0.8, 0.5, 0.2))

## 1.2 Target label 확인
- target label이 어떤 distribution(분포)을 가지고 있는지 확인해봐야 한다. 
- 지금 같은 binary classification 문제의 경우에서, 1과 0의 분포가 어떠냐에 따라 모델의 평가 방법이 달라질 수 있다.

In [11]:
f, ax = plt.subplots(1, 2, figsize=(18, 8))

df_train['Survived'].value_counts().plot.pie(explode=[0, 0.1], autopct='%1.1f%%', ax=ax[0], shadow=True)
ax[0].set_title('Pie plot - Survived')
ax[0].set_ylabel('')
sns.countplot('Survived', data=df_train, ax=ax[1])
ax[1].set_title('Count plot - Survived')

plt.show()

>파이 차트 : autopct는 부채꼴 안에 표시될 숫자 형식 / explode는 부채꼴이 파이 차트의 중심에서 벗어나는 정도 설정
- 안타깝게도 죽은 사람이 많다. 
- 38.4%가 살아남았다. 
- target label의 분포가 제법 균일(balanced)하다. 불균일한 경우, 예를 들어서 100 중 1이 99, 0이 1개인 경우에는 만약 모델이 모든 것을 1이라 해도 정확도가 99%가 나오게 된다. 0을 찾는 문제라면 이 모델은 원하는 결과를 줄 수 없게 된다. 지금 문제에서는 그렇지 않으니 계속 진행하자. 


## 2. Exploratory data analysis
- 이제 본격적으로 데이터 분석을 해보자. 데이터는 매우 많다. 이 많은 데이터 안에 숨겨진 사실을 찾기 위해선 적절한 시각화가 필요하다. 
- 시각화 라이브러리는 matplotlib, seaborn, plotly 등이 있다. 특정 목적에 맞는 소스코드를 정리해 필요할 때마다 참고하면 편하다. 
<br>

## 2.1 Pclass
- 먼저 Pclass에 대해 살펴보자. Pclass는 ordinal, 서수형 데이터다. 카테고리이면서, 순서가 있는 데이터 타입이다. 
- 먼저 Pclass에 따른 생존률의 차이를 살펴보자. 엑셀의 피벗 차트와 유사한 작업을 하게 되는데, pandas dataframe에서는 groupby를 사용하면 쉽게 할 수 있다. 또한, pivot이라는 메소드가 있다. 
- 'Pclass', 'Survived'를 가져온 후, pclass로 묶는다. 그러고 나면 각 pclass마다 0, 1이 count가 되는데, 이를 평균내면 각 pclass 별 생존률이 나온다. 
- 아래와 같이 count()를 하면, 각 class에 몇 명이 있는지 확인할 수 있으며, sum()을 하면, 216명 중 생존한(survived=1) 사람의 총합을 주게 된다. 

In [12]:
df_train[['Pclass', 'Survived']].groupby(['Pclass'], as_index=True).count()  #as_index : 그룹을 인덱스로 지정할 것인지 여부.

In [13]:
df_train[['Pclass', 'Survived']].groupby(['Pclass'], as_index=True).sum()

- pandas의 crosstab을 사용하면 좀 더 위 과정을 좀 더 수월하게 볼 수 있다. 

In [14]:
pd.crosstab(df_train['Pclass'], df_train['Survived'], margins=True).style.background_gradient(cmap='summer_r')

- grouped 객체에 mean()을 하게 되면, 각 클래스별 생존률을 얻을 수 있다. class 1이면 아래와 같다.  
$\frac{80}{(80+136)}\approx0.63$

In [15]:
df_train[['Pclass', 'Survived']].groupby(['Pclass'], as_index=True).mean().sort_values(by='Survived', ascending=False).plot.bar()

- 보다시피, Pclass가 좋을수록(1st) 생존률이 높은 것을 확인할 수 있다. 
- 좀 더 보기 쉽게 그래프를 그려보자. seaborn의 countplot을 이용하면, 특정 label에 따른 개수를 확인해 볼 수 있다. 

In [16]:
y_position = 1.02  # 타이틀이 그래프 위에 얼마나 떨어졌는지 정하는 상수 값.
f, ax = plt.subplots(1, 2, figsize=(18, 8))
df_train['Pclass'].value_counts().plot.bar(color=['#CD7F32', '#FFDF00', '#D3D3D3'], ax=ax[0])
ax[0].set_title('Number of Passengers By Pclass', y=y_position)
ax[0].set_ylabel('Count')
sns.countplot('Pclass', hue='Survived', data=df_train, ax=ax[1])
ax[1].set_title('Pclass: Survived vs Dead', y=y_position)
plt.show()

- 클래스가 높을수록, 생존 확률이 높은 것을 확인할 수 있다. Pclass 1, 2, 3 순서대로 63%, 48%, 25%다. 
- 우리는 생존에 Pclass가 큰 영향을 미친다고 생각해 볼 수 있으며, 나중에 모델을 세울 때 이 feature를 사용하는 것이 좋을 것이라 판단할 수 있다. 
<br>

## 2.2 Sex
- 이번에는 성별로 생존률이 어떻게 달라지는지 확인해보자.
- 마찬가지로 pandas groupby와 seaborn countplot을 사용해서 시각화해보자. 

In [17]:
f, ax = plt.subplots(1, 2, figsize=(18, 8))
df_train[['Sex', 'Survived']].groupby(['Sex'], as_index=True).mean().plot.bar(ax=ax[0])
ax[0].set_title('Survived vs Sex')
sns.countplot('Sex', hue='Survived', data=df_train, ax=ax[1])
ax[1].set_title('Sex: Survived vs Dead')
plt.show()

- 보다시피, 여자가 생존할 확률이 높다.

In [18]:
df_train[['Sex', 'Survived']].groupby(['Sex'], as_index=False).mean().sort_values(by='Survived', ascending=False)

In [19]:
pd.crosstab(df_train['Sex'], df_train['Survived'], margins=True).style.background_gradient(cmap='summer_r')

- Pclass와 마찬가지로, Sex도 예측 모델에 쓰일 중요한 feature임을 알 수 있다. 
<br>

## 2.3 Both Sex and Pclass
- 이번에는 Sex, Pclass 두 가지에 관하여 생존이 어떻게 달라지는지 확인해봅시다. 
- seaborn의 factorplot을 이용하면, 손쉽게 3개의 차원으로 이루어진 그래프를 그릴 수 있다. 

In [20]:
sns.factorplot('Pclass', 'Survived', hue='Sex', data=df_train, size=6, aspect=1.5)

- 모든 클래스에서 female이 살 확률이 male보다 높은 걸 알 수 있다. 
- 또한 남자, 여자 상관없이 클래스가 높을수록 살 확률이 높다.
- 위 그래프는 hue 대신 column으로 하면 아래와 같아진다.

In [21]:
sns.factorplot(x='Sex', y='Survived', col='Pclass', data=df_train, saturation=.5, size=9, aspect=1)

## 2.4 Age
- 이번에는 Age feature를 살펴보자. 

In [22]:
print('제일 나이 많은 탑승객 : {:.1f} Years'.format(df_train['Age'].max()))
print('제일 어린 탑승객 : {:.1f} Years'.format(df_train['Age'].min()))
print('탑승객 평균 나이 : {:.1f} Years'.format(df_train['Age'].mean()))      

- 생존에 따른 Age의 histogram을 그려보자.

In [23]:
fig, ax = plt.subplots(1, 1, figsize=(9, 5))
sns.kdeplot(df_train[df_train['Survived'] == 1]['Age'], ax=ax)
sns.kdeplot(df_train[df_train['Survived'] == 0]['Age'], ax=ax)
plt.legend(['Survived == 1', 'Survived == 0'])
plt.show()

>sns.kdeplot : 커널 밀도 추정 (히스토그램 같은 분포)
- 보다시피, 생존자 중 나이가 어린 경우가 많음을 볼 수 있다. 

In [24]:
# Age distribution withing classes
plt.figure(figsize=(8, 6))
df_train['Age'][df_train['Pclass'] == 1].plot(kind='kde')
df_train['Age'][df_train['Pclass'] == 2].plot(kind='kde')
df_train['Age'][df_train['Pclass'] == 3].plot(kind='kde')

plt.xlabel('Age')
plt.title('Age Distribution within classes')
plt.legend(['1st Class', '2nd Class', '3rd Class'])

- Class가 높을수록 나이 많은 사람의 비중이 커짐
- 나이대가 변하면서 생존률이 어떻게 되는지 보려고 한다.
- 나이 범위를 점점 넓혀가며, 생존률이 어떻게 되는지 한번 보자. 

In [25]:
cummulate_survival_ratio = []  # cummulate : 누적
for i in range(1, 80):
    cummulate_survival_ratio.append(df_train[df_train['Age'] < i]['Survived'].sum() / len(df_train[df_train['Age'] < i]['Survived']))
    
plt.figure(figsize=(7, 7))
plt.plot(cummulate_survival_ratio)
plt.title('Survival rate change depending on range of Age', y=1.02)
plt.ylabel('Survival rate')
plt.xlabel('Range of Age(0~x)')
plt.show()

- 보다시피, 나이가 어릴수록 생존률이 확실히 높은 것을 확인할 수 있다. 
- 우리는 이 나이가 중요한 feature로 쓰일 수 있음을 확인했다. 

## 2.5 Pclass, Sex, Age
- 지금까지 본 Sex, Pclass, Age, Survived 모두에 대해서 보고싶다. 이를 쉽게 그려주는 것이 seaborn의  violinplot이다. 
- x축은 우리가 나눠서 보고싶어하는 case(여기선 Pclass, Sex)를 나타내고, y축은 보고 싶어하는 distribution(Age)다. 
- 한 번 그려보자.

In [26]:
f, ax = plt.subplots(1, 2, figsize=(18, 8))
sns.violinplot("Pclass", "Age", hue="Survived", data=df_train, scale='count', split=True, ax=ax[0])
ax[0].set_title('Pclass and Age vs Survived')
ax[0].set_yticks(range(0, 110, 10))
sns.violinplot("Sex", "Age", hue="Survived", data=df_train, scale='count', split=True, ax=ax[1])
ax[1].set_title('Sex and Age vs Survived')
ax[1].set_yticks(range(0, 110, 10))
plt.show()

- 바이올린 플롯 : 중심선을 따라 대칭인 KDE 플롯이 있다. 
- 가운데 흰색 점은 중앙값(median)을 나타낸다.
- 바이올린 중앙의 두꺼운 선은 사분위 범위를 나타낸다. 
- 바이올린 중앙의 얇은 선은 신뢰 구간을 나타낸다. (95%)


- 왼쪽 그림은 Pclass 별로 Age의 distribution이 어떻게 다른지, 거기에 생존여부에 따라 구분한 그래프다. 
- 오른쪽 그림도 마찬가지 Sex, 생존에 따른 distribution이 어떻게 다른지 보여주는 그래프다. 
- 생존만 봤을 때, 모든 클래스에서 나이가 어릴수록 생존을 많이 한 것을 볼 수 있다. 
- 오른쪽 그림에서 보면, 명확히 여자가 생존을 많이 한 것을 볼 수 있다. 
- 여성과 아이를 먼저 챙긴 것을 볼 수 있다. 

## 2.6 Embarked
- Embarked는 탑승한 항구를 나타낸다.
- 위에서 해왔던 것과 비슷하게 탑승한 곳에 따른 생존률을 보자. 

In [27]:
f, ax = plt.subplots(1, 1, figsize=(7, 7))
df_train[['Embarked', 'Survived']].groupby(['Embarked'], as_index=True).mean().sort_values(by='Survived', ascending=False).plot.bar(ax=ax)

- 보다시피, 조금의 차이는 있지만 생존률은 좀 비슷한 것 같다. 그래도 C가 가장 높다.
- 모델에 얼마나 큰 영향을 미칠지는 모르겠지만, 그래도 사용하자. 
- 사실, 모델을 만들고 나면 우리가 사용한 feature들이 얼마나 중요한 역할을 했는지 확인해 볼 수 있다. 이는 추후에 모델을 만들고 난 다음에 살펴볼 것이다. 
- 다른 feature로 split해 한 번 살펴보자.

In [28]:
f, ax = plt.subplots(2, 2, figsize=(20, 15))
sns.countplot('Embarked', data=df_train, ax=ax[0, 0])
ax[0, 0].set_title('(1) No. Of Passengers Boarded')
sns.countplot('Embarked', hue='Sex', data=df_train, ax=ax[0, 1])
ax[0, 1].set_title('(2) Male-Female Split for Embarked')
sns.countplot('Embarked', hue='Survived', data=df_train, ax=ax[1, 0])
ax[1, 0].set_title('(3) Embarked vs Survived')
sns.countplot('Embarked', hue='Pclass', data=df_train, ax=ax[1, 1])
ax[1, 1].set_title('(4) Embarked vs Pclass')
plt.subplots_adjust(wspace=0.2, hspace=0.5)
plt.show()

- Figure(1) - 전체적으로 봤을 때, S에서 가장 많은 사람이 탑승했다. 
- Figure(2) - C와 Q는 남녀의 비율이 비슷하고,S는 남자가 더 많다.
- Figrue(3) - 생존확률이 S 경우 많이 낮은 걸 볼 수 있다. 
- Figure(4) - Class로 split해서 보니, C가 생존확률이 높은 건 클래스가 높은 건 클래스가 높은 사람이 많이 타서 그렇다. S는 3rd class가 많아서 생존확률이 낮게 나온다. 

## 2.7 Family - SipSp(형제 자매) + Parch(부모, 자녀)
- SipSp와 Parch를 합하면 Family가 될 것이다. Family로 합쳐서 분석해보자. 

In [29]:
df_train['FamilySize'] = df_train['SibSp'] + df_train['Parch'] + 1  # 자신을 포함해야하니 1을 더한다.
df_test['FamilySize'] = df_test['SibSp'] +df_test['Parch'] + 1  # 자신을 포함해야하니 1을 더한다. 

In [30]:
print("Maximum size of Family: ", df_train['FamilySize'].max())
print("Minimum size of Family: ", df_train['FamilySize'].min())

- FamliySize와 생존의 관계를 한 번 살펴보자. 

In [31]:
f, ax = plt.subplots(1, 3, figsize=(40, 10))
sns.countplot('FamilySize', data=df_train, ax=ax[0])
ax[0].set_title('(1) No. Of Passengers Boarded', y=1.02)

sns.countplot('FamilySize', hue='Survived', data=df_train, ax=ax[1])
ax[1].set_title('(2) Survived countplot depending on FamilySize', y=1.02)

df_train[['FamilySize', 'Survived']].groupby(['FamilySize'], as_index=True).mean().sort_values(by='Survived', ascending=False).plot.bar(ax=ax[2])
ax[2].set_title('(3) Survived rate depending on FamilySize', y=1.02)

plt.subplots_adjust(wspace=0.2, hspace=0.5)  # 서브 플롯 간의 간격 조정
plt.show()

- Figure (1) - 가족 크기가 1-11까지 있음을 볼 수 있다. 대부분 1명이고 그 다음으로 2, 3, 4명이다. 
- Figure (2), (3) - 가족 크기에 다른 생존비교다. 가족이 4명인 경우가 생존확률이 높다. 가족 수가 많아질수록, (5, 6, 7, 8, 11) 생존 확률이 낮아진다. 가족 수가 너무 작아도(1), 너무 커도(5, 6, 8, 11) 생존 확률이 작다. 3~4명 선에서 생존확률이 높은 걸 확인할 수 있다. 

## 2.8 Fare
- Fare는 탑승요금이며, continuous feature다. 한 번 histogram을 그려보자. 

In [32]:
fig, ax = plt.subplots(1, 1, figsize=(8, 8))
g = sns.distplot(df_train['Fare'], color='b', label='Skewness : {:.2f}'.format(df_train['Fare'].skew()), ax=ax)
g = g.legend(loc='best')

- 보다시피, distribution이 매우 비대칭인 것을 알 수 있다. (high skewness) 만약 이대로 모델에 넣어준다면 자칫 모델이 잘못 학습할 수도 있다. 몇 개 없는 outlier에 대해서 너무 민감하게 반응한다면, 실제 예측 시에 좋지 못한 결과를 부를 수 있다. 
- outlier의 영향을 줄이기 위해 Fare에 log를 취한다. 
- 여기서 우리는 pandas의 유용한 기능을 사용할 것이다. DataFrame의 특정 columns에 공통된 작업(함수)를 적용하고 싶으면, 아래의 map, 또는 apply를 사용하면 매우 손쉽게 적용할 수 있다. 
- 우리가 지금 원하는 것은 Fare columns의 데이터 모두를 log 값 취하는 것인데, 파이썬의 간단한 lambda 함수를 이용해 간단한 로그를 적용하는 함수를 map에 인수로 넣어주면, Fare columns 데이터에 그대로 적용된다. 매우 유용한 기능이니 꼭 숙지하자!

In [33]:
# 아래 줄은 뒤늦게 발견했다. 13번째 강의에 언급되니, 일단 따라치고 넘어가면 된다.
df_test.loc[df_test.Fare.isnull(), 'Fare'] = df_test['Fare'].mean()  # testset에 있는 nan value를 평균값으로 치환한다.

df_train['Fare'] = df_train['Fare'].map(lambda i: np.log(i) if i > 0 else 0)
df_test['Fare'] = df_test['Fare'].map(lambda i: np.log(i) if i > 0 else 0)

In [34]:
fig, ax = plt.subplots(1, 1, figsize=(8, 8))
g = sns.distplot(df_train['Fare'], color='b', label='Skewness : {:.2f}'.format(df_train['Fare'].skew()), ax=ax)
g = g.legend(loc='best')

- log를 취하니, 이제 비대칭성이 많이 사라진 것을 볼 수 있다. 
- 우리는 이런 작업을 사용해 모델이 좀 더 좋은 성능을 내도록 할 수 있다. 
- 사실 방금한 것은 feature engineering에 들어가는 부분인데, 여기서 작업했다.
- 모델을 학습시키기위해, 그리고 그 모델의 성능을 높이기 위해 feature들에 여러 조작을 가하거나, 새로운 feature를 추가하는 것을 feature engineering이라고 하는데, 우리는 이제 그것을 살펴볼 것이다. 

## 2.9 Cabin
- 이 feature는 NaN이 대략 80%이므로, 생존에 영향을 미칠 중요한 정보를 얻어내기가 쉽지는 않다. 
- 그러므로 우리가 세우려는 모델에 포함시키지 않도록 하자. 

In [35]:
df_train.head()

## 2.10 Ticket
- 이 feature는 NaN은 없다. 일단 string data이므로 우리가 어떤 작업들을 해주어야 실제 모델에 사용할 수 있는데, 이를 위해선 사실 아이디어가 필요하다. 

In [36]:
df_train['Ticket'].value_counts()

- 보다시피, ticket number는 매우 다양하다. 우리는 여기서 어떤 특징을 이끌어내서 생존과 연결시킬 수 있을까?
- 여러분이 직접 아이디어를 내보자. 이것이 본격적인 캐글 레이스의 시작점이다. 
- 이 튜토리얼에서는 튜토리얼이니 일단 ticket은 넘기도록 하자. 튜토리얼을 끝낸 후, 여러분의 모델의 성능을 향상시키기 위해 ticket에서 정보를 이끌어내는 것도 좋겠다. 
- 본 튜토리얼은 https://www.kaggle.com/arthurtok/introduction-to-ensembling-stacking-in-python, https://www.kaggle.com/startupsci/titanic-data-science-solutions, https://www.kaggle.com/ash316/eda-to-prediction-dietanic, https://www.kaggle.com/yassineghouzam/titanic-top-4-with-ensemble-modeling 을 참고해 만들었다. 공유해준 캐글러께 감사드린다. 